In [1]:
import os
import pandas as pd
from pydantic import SecretStr

from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA

from langchain.tools import tool, Tool
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

from environments import OPENAI_API_KEY
from prompts.rentabilidad_agricola import agricola_custom_prompt


In [2]:
if not OPENAI_API_KEY:
    raise ValueError("OpenAI Key not found")

openai_api_secret = SecretStr(OPENAI_API_KEY)

In [3]:
df = pd.read_csv("docs/datos_agricolas_3000_registros.csv")
df["Fecha"] = pd.to_datetime(df["Fecha"])

documentos = []
for _, row in df.iterrows():
    texto = f"""
    Fecha: {row['Fecha']}, Temporada: {row['Temporada']}, Cultivo: {row['Cultivo']}, Parcela: {row['Parcela']},
    Fase Productiva: {row['Fase Productiva']}, Cantidad (kg): {row['Cantidad (kg)']}, Precio por kg: {row['Precio por kg']},
    Ingresos: {row['Ingresos generados']}, Costos: {row['Costo Total']}, Rentabilidad: {row['Rentabilidad']},
    Clima: {row['Clima']}, Demanda de Mercado: {row['Demanda de Mercado']},
    Proyección Precio Temporada Siguiente: {row['Proyección Precio Temporada Siguiente']},
    Observaciones: {row['Observaciones']}, Línea Productiva: {row['Línea Productiva']}
    """.strip()
    documentos.append(texto)

# Preparación de los documentos para el RAG
docs = [Document(page_content=texto) for texto in documentos]

# Vectorización de los documentos con OpenAI
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_secret.get_secret_value())

persist_dir = "./chroma_rentabilidad"

if os.path.exists(persist_dir) and os.path.isdir(persist_dir):
    vectorstore = Chroma(persist_directory= persist_dir, embedding_function=embeddings)
else:
    vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=persist_dir)
    vectorstore.persist()

vectorstore = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_rentabilidad")

# Cargar vectorstore como retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

/tmp/ipykernel_160658/2525719845.py:28: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [4]:
# Crear chain de pregunta-respuesta
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4"),
    retriever=retriever,
    return_source_documents=True
)

In [5]:
pregunta = "¿Cual fue la rentabilidad más alta de la Parcela A en 2023?"
respuesta = qa_chain(pregunta)
respuesta

/tmp/ipykernel_160658/1227950309.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  respuesta = qa_chain(pregunta)


{'query': '¿Cual fue la rentabilidad más alta de la Parcela A en 2023?',
 'result': 'La rentabilidad más alta de la Parcela A en 2023 fue de 399.54.',
 'source_documents': [Document(metadata={}, page_content='Fecha: 2023-04-29 00:00:00, Temporada: 2023-Verano, Cultivo: Trigo, Parcela: Parcela A,\n    Fase Productiva: Cosecha, Cantidad (kg): 982, Precio por kg: 0.83,\n    Ingresos: 815.06, Costos: 494, Rentabilidad: 321.06,\n    Clima: Lluvioso, Demanda de Mercado: Alta,\n    Proyección Precio Temporada Siguiente: 0.95,\n    Observaciones: Actividad de cosecha bajo clima lluvioso, relacionado con parcela Parcela A., Línea Productiva: Grano'),
  Document(metadata={}, page_content='Fecha: 2023-04-29 00:00:00, Temporada: 2023-Verano, Cultivo: Trigo, Parcela: Parcela A,\n    Fase Productiva: Cosecha, Cantidad (kg): 982, Precio por kg: 0.83,\n    Ingresos: 815.06, Costos: 494, Rentabilidad: 321.06,\n    Clima: Lluvioso, Demanda de Mercado: Alta,\n    Proyección Precio Temporada Siguiente: 0.

In [6]:
pregunta = "¿Cual fue la parcela de mayor ingresos en el 2023?"
respuesta = qa_chain(pregunta)
respuesta

{'query': '¿Cual fue la parcela de mayor ingresos en el 2023?',
 'result': 'La parcela de mayor ingresos en el 2023 fue la Parcela D con ingresos de 486.39.',
 'source_documents': [Document(metadata={}, page_content='Fecha: 2023-04-06 00:00:00, Temporada: 2023-Verano, Cultivo: Maíz, Parcela: Parcela D,\n    Fase Productiva: Cosecha, Cantidad (kg): 523, Precio por kg: 0.93,\n    Ingresos: 486.39, Costos: 310, Rentabilidad: 176.39,\n    Clima: Soleado, Demanda de Mercado: Media,\n    Proyección Precio Temporada Siguiente: 0.85,\n    Observaciones: Actividad de cosecha bajo clima soleado, relacionado con parcela Parcela D., Línea Productiva: Grano'),
  Document(metadata={}, page_content='Fecha: 2023-04-06 00:00:00, Temporada: 2023-Verano, Cultivo: Maíz, Parcela: Parcela D,\n    Fase Productiva: Cosecha, Cantidad (kg): 523, Precio por kg: 0.93,\n    Ingresos: 486.39, Costos: 310, Rentabilidad: 176.39,\n    Clima: Soleado, Demanda de Mercado: Media,\n    Proyección Precio Temporada Siguient

In [7]:
pregunta = "¿Cuál fue la rentabilidad total de la Parcela A en 2023?"
respuesta = qa_chain(pregunta)
respuesta

{'query': '¿Cuál fue la rentabilidad total de la Parcela A en 2023?',
 'result': 'La rentabilidad total de la Parcela A en 2023 es la suma de las rentabilidades individuales. Según los datos proporcionados, las rentabilidades en 2023 fueron -334.0, -334.0, -416.0, -416.0, 321.06, 321.06, -421.0, -421.0, -179.0 y -179.0. Al sumar todos estos valores, obtenemos un total de -2858.88. Por lo tanto, la rentabilidad total de la Parcela A en 2023 fue de -2858.88.',
 'source_documents': [Document(metadata={}, page_content='Fecha: 2023-05-07 00:00:00, Temporada: 2023-Verano, Cultivo: Trigo, Parcela: Parcela A,\n    Fase Productiva: Mantenimiento, Cantidad (kg): 0, Precio por kg: 0.0,\n    Ingresos: 0.0, Costos: 334, Rentabilidad: -334.0,\n    Clima: Lluvioso, Demanda de Mercado: Alta,\n    Proyección Precio Temporada Siguiente: 0.95,\n    Observaciones: Actividad de mantenimiento bajo clima lluvioso, relacionado con parcela Parcela A., Línea Productiva: Grano'),
  Document(metadata={}, page_con

# Incorporación de Tools

In [8]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

# Tool: Rentabilidad por parcela
@tool
def rentabilidad_por_parcela(parcela: str, anio: int) -> str:
    """Calcula la rentabilidad total de una parcela en un año específico."""
    df = pd.read_csv("docs/datos_agricolas_3000_registros.csv")
    df["Fecha"] = pd.to_datetime(df["Fecha"])
    
    # Normalizar nombre de parcela si es "A" → "Parcela A"
    if not parcela.lower().startswith("parcela"):
        parcela = f"Parcela {parcela.upper()}"

    df_filtrado = df[(df["Parcela"] == parcela) & (df["Fecha"].dt.year == anio)]
    total = df_filtrado["Rentabilidad"].sum()
    return f"La rentabilidad total de la parcela {parcela} en {anio} fue de {total:.2f}"

# Tool: RAG como herramienta semántica
retriever_tool = Tool(
    name="consultar_datos_agricolas",
    func=qa_chain.invoke,
    description="Usa esto para buscar información general del cultivo, clima, observaciones o condiciones de mercado."
)

# Inicializar agente con ambas tools
tools = [rentabilidad_por_parcela, retriever_tool]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

/tmp/ipykernel_160658/490990612.py:27: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [9]:
# Probar una pregunta
pregunta = "¿Cuál fue la rentabilidad total de la Parcela A en 2023?"
respuesta = agent.invoke(pregunta)

# Mostrar la respuesta
print("Pregunta:", pregunta)
print("Respuesta:", respuesta)




> Entering new AgentExecutor chain...

Invoking: `rentabilidad_por_parcela` with `{'parcela': 'Parcela A', 'anio': 2023}`


La rentabilidad total de la parcela Parcela A en 2023 fue de 6776.89La rentabilidad total de la Parcela A en 2023 fue de 6776.89.

> Finished chain.
Pregunta: ¿Cuál fue la rentabilidad total de la Parcela A en 2023?
Respuesta: {'input': '¿Cuál fue la rentabilidad total de la Parcela A en 2023?', 'output': 'La rentabilidad total de la Parcela A en 2023 fue de 6776.89.'}


In [10]:
# Probar una pregunta
pregunta = "¿Cuál fue la parcela más rentable en 2023?"
respuesta = agent.invoke(pregunta)

# Mostrar la respuesta
print("Pregunta:", pregunta)
print("Respuesta:", respuesta)




> Entering new AgentExecutor chain...

Invoking: `rentabilidad_por_parcela` with `{'parcela': 'A', 'anio': 2023}`


La rentabilidad total de la parcela Parcela A en 2023 fue de 6776.89
Invoking: `rentabilidad_por_parcela` with `{'parcela': 'B', 'anio': 2023}`


La rentabilidad total de la parcela Parcela B en 2023 fue de -14034.90
Invoking: `rentabilidad_por_parcela` with `{'parcela': 'C', 'anio': 2023}`


La rentabilidad total de la parcela Parcela C en 2023 fue de 34361.80La parcela más rentable en 2023 fue la Parcela C con una rentabilidad total de 34361.80.

> Finished chain.
Pregunta: ¿Cuál fue la parcela más rentable en 2023?
Respuesta: {'input': '¿Cuál fue la parcela más rentable en 2023?', 'output': 'La parcela más rentable en 2023 fue la Parcela C con una rentabilidad total de 34361.80.'}


# Composición del Prompt con MarkDown

In [11]:
# Define custom prompt template for QA
qa_template = """
Basado en el siguiente contexto, responde a la pregunta de manera precisa:

Contexto:
{context}

Pregunta:
{question}

Respuesta:
"""

qa_prompt = PromptTemplate(
    template=qa_template,
    input_variables=["context", "question"]
)

# Tool: Rentabilidad por parcela
@tool
def rentabilidad_por_parcela(parcela: str, anio: int) -> str:
    """Calcula la rentabilidad total de una parcela en un año específico."""
    df = pd.read_csv("docs/datos_agricolas_3000_registros.csv")
    df["Fecha"] = pd.to_datetime(df["Fecha"])
    
    # Normalizar nombre de parcela si es "A" → "Parcela A"
    if not parcela.lower().startswith("parcela"):
        parcela = f"Parcela {parcela.upper()}"

    df_filtrado = df[(df["Parcela"] == parcela) & (df["Fecha"].dt.year == anio)]
    
    if df_filtrado.empty:
        return f"No se encontraron datos para la parcela {parcela} en el año {anio}."
    
    total = df_filtrado["Rentabilidad"].sum()
    return f"La rentabilidad total de la parcela {parcela} en {anio} fue de ${total:.2f}"

@tool
def porcentaje_costos_por_fase() -> str:
    """
    Retorna un análisis porcentual de los costos según las fases productivas.
    """
    df = pd.read_csv("docs/datos_agricolas_3000_registros.csv")
    
    # Agrupar por fase productiva y sumar costos
    costos_por_fase = df.groupby("Fase Productiva")["Costo Total"].sum()
    total_costos = costos_por_fase.sum()
    
    # Calcular porcentajes
    porcentajes = (costos_por_fase / total_costos * 100).round(2)

    # Formatear la respuesta
    resultado = "📊 Análisis porcentual de los costos por fase productiva:\n\n"
    for fase, porcentaje in porcentajes.items():
        resultado += f"• {fase}: {porcentaje:.2f}% del total de costos\n"

    return resultado

@tool
def analisis_demanda_mercado() -> str:
    """
    Realiza un análisis de la demanda de mercado mostrando la relación con la rentabilidad,
    el precio promedio por kg y el volumen total en kg.
    """
    df = pd.read_csv("docs/datos_agricolas_3000_registros.csv")

    # Filtrar datos válidos
    df = df.dropna(subset=["Demanda de Mercado", "Rentabilidad", "Precio por kg", "Cantidad (kg)"])

    # Agrupar por nivel de demanda
    resumen = df.groupby("Demanda de Mercado").agg({
        "Rentabilidad": "mean",
        "Precio por kg": "mean",
        "Cantidad (kg)": "sum"
    }).round(2)

    # Formatear el resultado
    resultado = "📈 Análisis de la demanda de mercado:\n\n"
    for demanda, row in resumen.iterrows():
        resultado += (
            f"• Demanda **{demanda}**:\n"
            f"   - Rentabilidad promedio: ${row['Rentabilidad']:.2f}\n"
            f"   - Precio promedio por kg: ${row['Precio por kg']:.2f}\n"
            f"   - Volumen total: {int(row['Cantidad (kg)'])} kg\n\n"
        )

    return resultado

# Tool: RAG como herramienta semántica
def consultar_datos_agricolas(query):
    """Usa esto para buscar información general del cultivo, clima, observaciones o condiciones de mercado."""
    result = qa_chain.invoke({"query": query})
    # Extraer solo la respuesta sin los documentos de origen para la salida
    return result['result']

retriever_tool = Tool(
    name="consultar_datos_agricolas",
    func=consultar_datos_agricolas,
    description="Usa esto para buscar información general del cultivo, clima, observaciones o condiciones de mercado."
)

# Definir el template exacto que se proporcionó
custom_markdown_template = agricola_custom_prompt

# Inicializar agente con las herramientas y el template de markdown personalizado
tools = [rentabilidad_por_parcela, retriever_tool, porcentaje_costos_por_fase, analisis_demanda_mercado]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs={
        "system_message": custom_markdown_template
    }
)



In [12]:
pregunta_rentabilidad = "¿Cuál fue la rentabilidad total de la Parcela A en 2023?"
respuesta = agent.invoke(f"¿{pregunta_rentabilidad} y cómo se comparó con el clima de ese año?")

print("Pregunta:", pregunta)
print("Respuesta:", respuesta)




> Entering new AgentExecutor chain...

Invoking: `rentabilidad_por_parcela` with `{'parcela': 'Parcela A', 'anio': 2023}`


La rentabilidad total de la parcela Parcela A en 2023 fue de $6776.89
Invoking: `consultar_datos_agricolas` with `clima en 2023`


Based on the given context, we can see that there were two types of weather reported: rainy and sunny. However, without date-specific information, it's not possible to say what the weather was for the whole year of 2023.La rentabilidad total de la Parcela A en 2023 fue de $6776.89.

En cuanto al clima de ese año, se reportaron dos tipos de condiciones climáticas: lluviosas y soleadas. Sin embargo, sin información específica de fechas, no es posible determinar cómo fue el clima durante todo el año 2023.

> Finished chain.
Pregunta: ¿Cuál fue la parcela más rentable en 2023?
Respuesta: {'input': '¿¿Cuál fue la rentabilidad total de la Parcela A en 2023? y cómo se comparó con el clima de ese año?', 'output': 'La rentabilidad total de la

In [13]:
pregunta_rentabilidad = "Quiero un análisis comparativo de la rentabilidad para cada una de las parcerlas"
respuesta = agent.invoke(f"{pregunta_rentabilidad}")

print("Pregunta:", pregunta)
print("Respuesta:", respuesta)



> Entering new AgentExecutor chain...
Claro, para poder ayudarte necesito que me proporciones la lista de las parcelas que quieres analizar y el año específico para el que necesitas la información.

> Finished chain.
Pregunta: ¿Cuál fue la parcela más rentable en 2023?
Respuesta: {'input': 'Quiero un análisis comparativo de la rentabilidad para cada una de las parcerlas', 'output': 'Claro, para poder ayudarte necesito que me proporciones la lista de las parcelas que quieres analizar y el año específico para el que necesitas la información.'}


In [14]:
pregunta_costos = "Cómo se distribuyen los costos entre las fases productivas"
respuesta = agent.invoke(f"{pregunta_costos}?")

print("Pregunta:", pregunta)
print("Respuesta:", respuesta)



> Entering new AgentExecutor chain...

Invoking: `porcentaje_costos_por_fase` with `{}`


📊 Análisis porcentual de los costos por fase productiva:

• Cosecha: 32.86% del total de costos
• Mantenimiento: 34.00% del total de costos
• Siembra: 33.14% del total de costos
La distribución de los costos entre las fases productivas es la siguiente:

- La fase de **cosecha** representa el 32.86% del total de costos.
- La fase de **mantenimiento** representa el 34.00% del total de costos.
- La fase de **siembra** representa el 33.14% del total de costos.

Esto significa que los costos están bastante equilibrados entre las tres fases, con un ligero predominio en la fase de mantenimiento.

> Finished chain.
Pregunta: ¿Cuál fue la parcela más rentable en 2023?
Respuesta: {'input': 'Cómo se distribuyen los costos entre las fases productivas?', 'output': 'La distribución de los costos entre las fases productivas es la siguiente:\n\n- La fase de **cosecha** representa el 32.86% del total de costos.\

In [15]:
pregunta_demanda = "¿Cuál fue el volumen vendido y la rentabilidad según el nivel de demanda?"
respuesta = agent.invoke(f"{pregunta_demanda}?")

print("Pregunta:", pregunta)
print("Respuesta:", respuesta)



> Entering new AgentExecutor chain...

Invoking: `analisis_demanda_mercado` with `{}`


📈 Análisis de la demanda de mercado:

• Demanda **Alta**:
   - Rentabilidad promedio: $15.95
   - Precio promedio por kg: $0.61
   - Volumen total: 198530 kg

• Demanda **Baja**:
   - Rentabilidad promedio: $-22.96
   - Precio promedio por kg: $0.55
   - Volumen total: 195705 kg

• Demanda **Media**:
   - Rentabilidad promedio: $36.86
   - Precio promedio por kg: $0.60
   - Volumen total: 191041 kg

Aquí está el análisis de la demanda de mercado:

- Cuando la demanda es **alta**:
  - La rentabilidad promedio es de $15.95.
  - El precio promedio por kg es de $0.61.
  - El volumen total vendido es de 198,530 kg.

- Cuando la demanda es **baja**:
  - La rentabilidad promedio es de $-22.96, lo que indica una pérdida.
  - El precio promedio por kg es de $0.55.
  - El volumen total vendido es de 195,705 kg.

- Cuando la demanda es **media**:
  - La rentabilidad promedio es de $36.86.
  - El precio prome